In [59]:
import tensorflow as tf
from tensorflow.keras import layers
import pickle
import numpy as np
import cv2
import random

In [60]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

def randomShowPics():
    info = unpickle('dataset/batches.meta')
    label_names = info['label_names']
    data = unpickle('dataset/data_batch_1')
    imgs = data['data'].reshape(-1,3,32,32)
    labels = data['labels']
    imgs = np.transpose(imgs, (0, 2, 3,1))
    for i in range(10):
        idx = random.randint(0,9999)
        img = imgs[idx]
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imshow(label_names[labels[idx]],img)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()
        
def loadDataset():
    data = unpickle('dataset/data_batch_1')
    X_train = data['data'].reshape(-1,3,32,32)
    Y_train = np.array(data['labels']).reshape(-1,1)
    for i in range(2,6):
        data = unpickle('dataset/data_batch_'+str(i))
        X = data['data'].reshape(-1,3,32,32)
        Y = np.array(data['labels']).reshape(-1,1)
        X_train = np.concatenate((X_train,X), axis=0)
        Y_train = np.concatenate((Y_train,Y), axis=0)
    X_train = np.transpose(X_train, (0, 2, 3, 1)) 
    
    data_test = unpickle('dataset/test_batch')
    X_test = data_test['data'].reshape(-1,3,32,32)
    X_test = np.transpose(X_test, (0, 2, 3, 1)) 
    Y_test = np.array(data_test['labels']).reshape(-1,1)
    return X_train, Y_train, X_test, Y_test
    
def LeNetModel():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(filters=6, kernel_size=(5, 5),strides=(1, 1), padding='valid',activation='relu', input_shape=(32,32,3)))
    model.add(layers.MaxPool2D(pool_size=(2, 2),padding='valid'))
    model.add(layers.Conv2D(filters=16, kernel_size=(5, 5),strides=(1, 1), padding='valid',activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2, 2),padding='valid'))
    model.add(layers.Flatten())
    model.add(layers.Dense(120,activation='relu'))
    model.add(layers.Dense(84,activation='relu'))
    model.add(layers.Dense(10,activation='softmax'))
    model.summary()
    return model
    
def train(X_train, Y_train, X_test, Y_test,model):
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train, Y_train, epochs=10,batch_size=16)
    loss, acc = model.evaluate(x=X_test, y=Y_test)
    print("loss: %s, acc: %s" % (loss, acc))
    


In [57]:

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [61]:
X_train, Y_train, X_test, Y_test = loadDataset()
lenet_model = LeNetModel()
train(X_train, Y_train, X_test, Y_test, lenet_model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         456       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at /home/luben/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_978]

Function call stack:
distributed_function


In [62]:
tf.__version__

'2.0.0'